# Swarm Algorithm for Objects Moving in 2D Domain
The swarm concept in recents years gained quite popularity in many applications, especially in defence industry. The most attracting side of swarm concept is to be able to fulfill complex duties with relatively cheap ammo.

The code below shows swarming particles (like drones) in 2 Dimensional domain and has been generated with rule based algorithm given in the great paper "Outdoor flocking and formation flight with autonomous aerial robots" by Vicsek et. al. [1] The document referenced in [2] is also a very explanatory and beneficial to understand the basics of swarm concept.

The first thing to do is to import necessary packages to our environment. 

# Refences
[1] "Outdoor flocking and formation flight with autonomous aerial robots", G. Vásárhelyi, Cs. Virágh, G. Somorjai, N. Tarcai, T. Szörényi, T. Nepusz, T. Vicsek

[2] "Dynamic Mission Control for UAV Swarm via Task Stimulus Approach", Haoyang Cheng , John Page, John Olsen

[3] https://github.com/ibrahimkaya754/GeneticAlgorithm

In [1]:
# Import neccessary modules
import time
import math
import matplotlib.pyplot as plt
import pygame
import random
import numpy as np
from pygame.locals import *
from PygameModule import *
from Swarm_Algorithm_RuleBased import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


What is next is, we are deciding the values of some simulation parameters, like the time step size, number of particles swarming, total run time, etc.

In [2]:
# Simulation Parameters
number_of_particles = 51
number_of_axes      = 2
delta_t             = 0.1
t_final             = 1000
wght_leader         = np.array((0.0,0.0,0.0,0.0,10.0,0.0))

# Main Parameters
The parameters given below inside the main function has been obtained by Genetic Algorithm (GA). The writers of the paper did not give the values of the parameters, which is quite reasonable to some point, since every application has its own dynamics, i.e., the parameters from one applicastion to the other will probably be very different. For my application, I used GA code I have shared in my github repo [3] and the algorithm has given the parameters seen inside the main function.

There are 2 important modules imported to our environment above. One of them is PygameModule which generates the neccessary things for visualization of the particles moving with swarm condition. The other one is the Swarm_Algoritm_RuleBased, where the rulebased algorithm given in the paper is coded.

What is happenning in the simulation can be summarized as the following:

* There are 51 particles swarming in the domain

* One of them is the leader that every information about the mission is known by

* The other 50 particles does not know anything about the mission but the only thing they know is they should follow the leader without creating any crash inside the population

* The points of the target that is to be followed were given in the list of waypoints, where it can be changed inside the domain randomly.

* The parameters obtained by GA has been generated for a fitness function minimizing the sum of the distances between the group members (minimization of the dispersion), while preventing the individual particles from approaching to eachother greater than a value defined (8 meters given in our case).

* Although the number of total particles (population) has been given as 51, it can be increased (to 100s) or decreased (to around 10) since the algorithm coded can be applied for any number of particles, i.e., the algorithm given here is scalable.

In [3]:
best_position_ever =  {'0': 19.087, 
                       '1': 77.570, 
                       '2': 74.741, 
                       '3': 49.385, 
                       '4': 50.461, 
                       '5': 31.121, 
                       '6': 715.096, 
                       '7': 87.658, 
                       '8': -8.527, 
                       '9': 9.441, 
                       '10': -1.126, 
                       '11': 7.908, 
                       '12': 5.326, 
                       '13': -3.060}

In [4]:
params_best = [param for param in best_position_ever.values()]

In [6]:
# main function
def main(paramaters=[19.087,77.570,74.741,49.385,50.461,31.121,715.096,
                     87.658,-8.527,9.441,-1.126,7.908,5.326,-3.060],TimeConstant=0.1):
    wght_fllwr               = np.array((paramaters[8],paramaters[9],paramaters[10],
                                         paramaters[11],paramaters[12],paramaters[13]),dtype='double')
    distances                = np.zeros((number_of_particles,number_of_particles*number_of_axes))
    distances_abs            = np.zeros((number_of_particles,number_of_particles))
    closest_distances_abs    = np.zeros((number_of_particles,number_of_particles))
    closest_particles_abs    = np.zeros((number_of_particles,number_of_particles),dtype='int32')
    closestneighbours        = np.zeros((number_of_particles),dtype='int16')
    position                 = np.zeros((number_of_particles,number_of_axes)) 
    velocity                 = np.zeros((number_of_particles,number_of_axes)) 
    position_delta           = np.zeros((number_of_particles,number_of_axes))
    dist_twp                 = np.zeros((number_of_particles-1,1))

    screen_size = [3000, 1300]
    screen = pygame.display.set_mode(tuple(screen_size))
    pygame.display.set_caption("Swarm")
    
#     xtrg              = [2200.0,650.0] + np.round(np.multiply(np.subtract(screen_size,[2700,1200]),[np.random.random()]))
    xtrg              = [2400,200]
    list_min_distance = []
    list_ave_distance = []
            
    background = pygame.Surface(screen.get_size())
    background.fill((255, 255, 255))
    screen.blit(background, (0, 0))
    
    particles = np.zeros((number_of_particles),dtype = particle)
    for ii in range(number_of_particles-2):
        particles[ii]  = particle(screen, background, color =0)
        position[ii,0] = particles[ii].positionx
        position[ii,1] = particles[ii].positiony
        velocity[ii,0] = particles[ii].velx
        velocity[ii,1] = particles[ii].vely
    for ii in range(number_of_particles-2,number_of_particles-1):
        particles[ii]  = particle(screen, background, color =1)
        position[ii,0] = particles[ii].positionx
        position[ii,1] = particles[ii].positiony
        velocity[ii,0] = particles[ii].velx
        velocity[ii,1] = particles[ii].vely
    for ii in range(number_of_particles-1,number_of_particles):
        particles[ii]  = particle(screen, background, color =2)
        particles[ii].positionx = xtrg[0]
        particles[ii].positiony = xtrg[1]
        position[ii,0] = particles[ii].positionx
        position[ii,1] = particles[ii].positiony
        velocity[ii,0] = particles[ii].velx
        velocity[ii,1] = particles[ii].vely
    
    dist = distance(population_number=number_of_particles,dimension=number_of_axes,
                    distances=distances,distances_abs=distances_abs,
                    position=position,
                    closest_distances_abs=closest_distances_abs,closest_particles_abs=closest_particles_abs,
                    closestneighbours=closestneighbours)
    
    distances,distances_abs,closest_distances_abs,closest_particles_abs,closestneighbours = dist.find_distances()
    swarm_algo_follower = swarm_algorithm(params=paramaters)
    swarm_algo_leader   = swarm_algorithm(params=paramaters)
    
    allSprites = pygame.sprite.Group(particles[:]) # Grouping the objects to use the uniform method
    clock = pygame.time.Clock()
    time1 = time.process_time()
    keepGoing = True
    iter      = 0
    t         = 0
    counter   = 0

    while keepGoing:
        try:
            xtrg      = np.add(xtrg,np.multiply([4,4],0.01))
            remainder = iter % 100        
            for ii in range(number_of_particles-2):
                dist_to_ldr = dist.dist_to_wypnt(position[ii],position[number_of_particles-2])
                for jj in range(number_of_axes):
                    swarm_algo_follower.algo(particle=ii,axes=jj,position=position,velocity=velocity,
                                             closest_particles_abs=closest_particles_abs,delta_t=delta_t,
                                             xtrg=position[number_of_particles-2],TimeConstant=TimeConstant,
                                             wght=wght_fllwr,distance_to_target=dist_to_ldr)
                    position_delta[ii,jj] = swarm_algo_follower.position_delta
                    velocity[ii,jj]       = swarm_algo_follower.velocity[ii,jj]
                
                particles[ii].dx = position_delta[ii,0]
                particles[ii].dy = position_delta[ii,1]
                
                particles[ii].update()
                position[ii,0] = particles[ii].positionx
                position[ii,1] = particles[ii].positiony                    
            
            for ii in range(number_of_particles-2,number_of_particles-1):
                dist_to_trg = dist.dist_to_wypnt(position[ii],xtrg)
                for jj in range(number_of_axes): 
                    swarm_algo_leader.algo(particle=ii,axes=jj,position=position,velocity=velocity,delta_t=delta_t,
                                           closest_particles_abs=closest_particles_abs,TimeConstant=TimeConstant, 
                                           xtrg=xtrg,wght=wght_leader,distance_to_target=dist_to_trg)
                    
                    position_delta[ii,jj] = swarm_algo_leader.position_delta  
                    velocity[ii,jj]       = swarm_algo_leader.velocity[ii,jj]
    
                particles[ii].dx = position_delta[ii,0]
                particles[ii].dy = position_delta[ii,1]
                particles[ii].update()
                position[ii,0] = particles[ii].positionx
                position[ii,1] = particles[ii].positiony       
                              
            particles[number_of_particles-1].dx          = 0
            particles[number_of_particles-1].dy          = 0
            particles[number_of_particles-1].update()
            particles[number_of_particles-1].positionx   = xtrg[0]
            particles[number_of_particles-1].positiony   = xtrg[1]                
            particles[number_of_particles-1].rect.center = (particles[number_of_particles-1].positionx, 
                                                            particles[number_of_particles-1].positiony)
            
            position[number_of_particles-1,0]            = particles[number_of_particles-1].positionx
            position[number_of_particles-1,1]            = particles[number_of_particles-1].positiony
            
            distances,distances_abs,closest_distances_abs, \
            closest_particles_abs,closestneighbours = dist.find_distances()
            dist_to_trg = dist.dist_to_wypnt(position[number_of_particles-2],xtrg)
            
            iter = iter + 1
            t    = t + delta_t
            counter = counter + 1
            list_min_distance.append(np.min(closest_distances_abs[0:98,1]))
            list_ave_distance.append(np.average(closest_distances_abs[0:98,1:100]))
            
            if dist_to_trg <= 5.0:
                if np.random.rand() >= 0.98:
                    xtrg      = [50.0,50.0] + \
                                np.round(np.multiply(np.subtract(screen_size,[100,100]),[np.random.random()]))
            
            if t >= t_final:
                keepGoing = False
            
            if remainder == 0:
                print('time = ',t,' s ', ' target_pos = ', xtrg)
                print('average_min_distance = ', np.average(list_min_distance))
                print('average_group_distance = ', np.average(list_ave_distance))
                print('\n')
                counter = 0
#################################################################################################################################
            allSprites.clear(screen, background)
            allSprites.draw(screen)\

            pygame.display.flip()
        except:
            print('unexpected error --- sorry')
            keepGoing = False
            
    pygame.quit()
    time2 = time.process_time()
    delta_time = time2 - time1
    return position,velocity,distances,closestneighbours,delta_time, \
           closest_particles_abs, closest_distances_abs, list_min_distance, list_ave_distance

Below is the cell where the above code is run. 

Although I have run the code for a simulation time of 100 seconds, one can run it as long as he likes.

In [7]:
# run
if __name__ == "__main__":
    position,velocity,distances,closestneighbours,delta_time, \
    closest_particles_abs,closest_distances_abs, list_min_distance, list_ave_distance = main(params_best,TimeConstant=0.0001)

time =  0.1  s   target_pos =  [2400.04  200.04]
average_min_distance =  4.412241687360311
average_group_distance =  435.17345758189373


time =  10.09999999999998  s   target_pos =  [2404.04  204.04]
average_min_distance =  4.846999589084988
average_group_distance =  438.9026846872932


time =  20.100000000000016  s   target_pos =  [2408.04  208.04]
average_min_distance =  9.75433046084003
average_group_distance =  443.00092977003817


time =  30.100000000000158  s   target_pos =  [2412.04  212.04]
average_min_distance =  11.567578040422287
average_group_distance =  446.1705606928886


time =  40.1000000000003  s   target_pos =  [2416.04  216.04]
average_min_distance =  12.476462737419876
average_group_distance =  449.2177183874468


unexpected error --- sorry


PygameModule.particle

In [ ]:
velocity

In [ ]:
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance)

In [ ]:
list_ave_distance_kinetic_50 = list_ave_distance

In [ ]:
plt.figure(figsize=(26,9))
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinematic,label='kinematic')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_01,label='TimeConstant=0.01')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_02,label='TimeConstant=0.02')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_03,label='TimeConstant=0.03')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_05,label='TimeConstant=0.05')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_10,label='TimeConstant=0.10')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_15,label='TimeConstant=0.15')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_20,label='TimeConstant=0.20')
plt.plot(np.arange(len(list_ave_distance)),list_ave_distance_kinetic_50,label='TimeConstant=0.50')
plt.legend()
plt.show()

In [1]:
'''
Author: ikaya
'''
#%%
# Import neccessary modules
import time
import math
import matplotlib.pyplot as plt
import pygame
import random
import numpy as np
from pygame.locals import *
from swarm import *
from pso.PSO import swarm as opt

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
################################################################################################
#%%
# Simulation Parameters
number_of_particles = 30
number_of_axes      = 2
delta_t             = 0.2
t_final             = 100
screen_size         = [3000,1800]
initial_location    = [screen_size[0]/2,screen_size[1]/2]
list_min_distance   = []
list_ave_distance   = []
xtrg                = [initial_location[ii] + np.random.randint([900,1400])[ii] for ii in range(number_of_axes)]
particles           = swarm(number_of_particles=number_of_particles, screensize=screen_size, target_location=xtrg,
                          display=True, CommRng=100, dim=number_of_axes, delta_t= delta_t)
leader              = particles.leader
numberofneighbour   = 2
numberofleader      = 1
clock               = pygame.time.Clock()
numberofepochs      = 1

### The multiplayer 2 below is for 'position' and 'velocity' ###
print('----------------------------------------------------------------------------')
print('There will be %s states, %s for relative velocity, %s for relative position' % \
      (particles.dim*(numberofneighbour+numberofleader)*2,\
       particles.dim*(numberofneighbour+numberofleader),\
       particles.dim*(numberofneighbour+numberofleader)))
print('----------------------------------------------------------------------------')


-----------------------
SUMMARY FOR PARTICLES
-----------------------

particle id :  0
role        :  leader
target      :  target
wghts       :  [ 0.  0.  0.  0. 10.  0.]
dist2wp     :  1614.2600917685581
-----------------------
particle id :  1
role        :  follower
target      :  leader
wghts       :  [-8.527, 9.441, -1.126, 7.908, 5.326, -3.06]
dist2wp     :  323.94548667506103
-----------------------
particle id :  2
role        :  follower
target      :  leader
wghts       :  [-8.527, 9.441, -1.126, 7.908, 5.326, -3.06]
dist2wp     :  201.4481786469484
-----------------------
particle id :  3
role        :  follower
target      :  leader
wghts       :  [-8.527, 9.441, -1.126, 7.908, 5.326, -3.06]
dist2wp     :  274.6672086690322
-----------------------
particle id :  4
role        :  follower
target      :  leader
wghts       :  [-8.527, 9.441, -1.126, 7.908, 5.326, -3.06]
dist2wp     :  163.94264679600641
-----------------------
particle id :  5
role        :  follower
targe

In [3]:
################################################################################################
#%%
def function2opt(params):
    particles.import_coefficients = True
    particles.params = params
    t                = 0 
    mean_closest     = np.zeros(number_of_particles)
    mean_furthest    = np.zeros(number_of_particles)
    mean2target      = np.zeros(number_of_particles)
    val2min          = []
    
    while t<=t_final:
        particles.rulebasedalgo()
        particles.update(keepGoing=True)
        print('time: %.2f' % (t))
        t = t + delta_t
    
        if t%t_final >= 0.0 and t%t_final < delta_t:
            print('\ntarget location changes\n')
            particles.trgt_loc                 = {str(ii) : np.random.randint(screen_size)[ii] for ii in range(particles.dim)}
            particles.targetposition['target'] = particles.trgt_loc
    
        for ii in range(particles.nop):
            mean_closest[ii]  = np.abs(150.0-particles.member[str(ii)]['abs_distance_sorted'][1])
            mean_furthest[ii] = np.abs(750.0-particles.member[str(ii)]['abs_distance_sorted'][-1])
            mean2target[ii]   = particles.member[str(ii)]['distance2target']
            
        val2min.append(4*np.sum(mean_closest) + np.sum(mean_furthest)+ np.sum(mean2target))
        
    return np.sum(val2min)

In [ ]:
################################################################################################
#%%
xtrg         = [np.random.randint(screen_size)[ii] for ii in range(number_of_axes)]
optimization = opt(func=function2opt,lowerbounds=-100*np.ones(14),upperbounds=100*np.ones(14),number_of_particles=60,readsavedfile=[True,'best_vals_ever.txt'])
particles.__init__(number_of_particles=number_of_particles, screensize=screen_size, target_location=xtrg, display=True, CommRng=100, summary=False)

time: 0.00
time: 0.20
time: 0.40
time: 0.60
time: 0.80
time: 1.00
time: 1.20
time: 1.40
time: 1.60
time: 1.80
time: 2.00
time: 2.20
time: 2.40
time: 2.60
time: 2.80
time: 3.00
time: 3.20
time: 3.40
time: 3.60
time: 3.80
time: 4.00
time: 4.20
time: 4.40
time: 4.60
time: 4.80
time: 5.00
time: 5.20
time: 5.40
time: 5.60
time: 5.80
time: 6.00
time: 6.20
time: 6.40
time: 6.60
time: 6.80
time: 7.00
time: 7.20
time: 7.40
time: 7.60
time: 7.80
time: 8.00
time: 8.20
time: 8.40
time: 8.60
time: 8.80
time: 9.00
time: 9.20
time: 9.40
time: 9.60
time: 9.80
time: 10.00
time: 10.20
time: 10.40
time: 10.60
time: 10.80
time: 11.00
time: 11.20
time: 11.40
time: 11.60
time: 11.80
time: 12.00
time: 12.20
time: 12.40
time: 12.60
time: 12.80
time: 13.00
time: 13.20
time: 13.40
time: 13.60
time: 13.80
time: 14.00
time: 14.20
time: 14.40
time: 14.60
time: 14.80
time: 15.00
time: 15.20
time: 15.40
time: 15.60
time: 15.80
time: 16.00
time: 16.20
time: 16.40
time: 16.60
time: 16.80
time: 17.00
time: 17.20
time: 

time: 38.00
time: 38.20
time: 38.40
time: 38.60
time: 38.80
time: 39.00
time: 39.20
time: 39.40
time: 39.60
time: 39.80
time: 40.00
time: 40.20
time: 40.40
time: 40.60
time: 40.80
time: 41.00
time: 41.20
time: 41.40
time: 41.60
time: 41.80
time: 42.00
time: 42.20
time: 42.40
time: 42.60
time: 42.80
time: 43.00
time: 43.20
time: 43.40
time: 43.60
time: 43.80
time: 44.00
time: 44.20
time: 44.40
time: 44.60
time: 44.80
time: 45.00
time: 45.20
time: 45.40
time: 45.60
time: 45.80
time: 46.00
time: 46.20
time: 46.40
time: 46.60
time: 46.80
time: 47.00
time: 47.20
time: 47.40
time: 47.60
time: 47.80
time: 48.00
time: 48.20
time: 48.40
time: 48.60
time: 48.80
time: 49.00
time: 49.20
time: 49.40
time: 49.60
time: 49.80
time: 50.00
time: 50.20
time: 50.40
time: 50.60
time: 50.80
time: 51.00
time: 51.20
time: 51.40
time: 51.60
time: 51.80
time: 52.00
time: 52.20
time: 52.40
time: 52.60
time: 52.80
time: 53.00
time: 53.20
time: 53.40
time: 53.60
time: 53.80
time: 54.00
time: 54.20
time: 54.40
time

time: 75.60
time: 75.80
time: 76.00
time: 76.20
time: 76.40
time: 76.60
time: 76.80
time: 77.00
time: 77.20
time: 77.40
time: 77.60
time: 77.80
time: 78.00
time: 78.20
time: 78.40
time: 78.60
time: 78.80
time: 79.00
time: 79.20
time: 79.40
time: 79.60
time: 79.80
time: 80.00
time: 80.20
time: 80.40
time: 80.60
time: 80.80
time: 81.00
time: 81.20
time: 81.40
time: 81.60
time: 81.80
time: 82.00
time: 82.20
time: 82.40
time: 82.60
time: 82.80
time: 83.00
time: 83.20
time: 83.40
time: 83.60
time: 83.80
time: 84.00
time: 84.20
time: 84.40
time: 84.60
time: 84.80
time: 85.00
time: 85.20
time: 85.40
time: 85.60
time: 85.80
time: 86.00
time: 86.20
time: 86.40
time: 86.60
time: 86.80
time: 87.00
time: 87.20
time: 87.40
time: 87.60
time: 87.80
time: 88.00
time: 88.20
time: 88.40
time: 88.60
time: 88.80
time: 89.00
time: 89.20
time: 89.40
time: 89.60
time: 89.80
time: 90.00
time: 90.20
time: 90.40
time: 90.60
time: 90.80
time: 91.00
time: 91.20
time: 91.40
time: 91.60
time: 91.80
time: 92.00
time

In [ ]:
################################################################################################
#%%
optimization.update(iteration=2)
#%%  Save the best values
best_vals_list = []
for elem in list(optimization.best_position.keys()): 
    best_vals_list.append(list(optimization.member[elem]['best_position'].values()))

best_vals_ever_list = []
for elem in list(optimization.best_position_ever.keys()): 
    best_vals_ever_list.append(list(optimization.member[elem]['best_position'].values()))
    
np.savetxt('best_vals.txt',best_vals_list)
np.savetxt('best_vals_ever.txt',best_vals_ever_list)

In [ ]:
################################################################################################
#%%                
for key in particles.member.keys():
    print('Particle id       : %s' % (key))
    print('Particle role     : %s' % (particles.member[key]['role']))
    print('Particle color    : ', particles.color[particles.member[key]['role']])
    print('Particle target   : %s' % (particles.member[key]['target']))
    print('particle velocity : %s' % (particles.member[key]['velocity']))
    print('particle position : %s' % (particles.member[key]['position']))
    print('target position   : %s' % (particles.targetposition[particles.member[key]['target']]))
    print('weigths           : %s' % (particles.wght[particles.member[key]['role']]))
    print('particles in rng  : %s' % (particles.member[key]['PrtclsInRng']))
    print('------------------------------------')
################################################################################################
# %%
print('best_particle      : ',list(optimization.best_position.keys())[0])
print('best_position      : ',optimization.member[list(optimization.best_position.keys())[0]]['best_position'])
print('best_value         : ',optimization.member[list(optimization.best_position.keys())[0]]['fitness_value'])
print('best_particle_ever : ',list(optimization.best_position_ever.keys())[0])
print('best_position_ever : ',optimization.member[list(optimization.best_position_ever.keys())[0]]['best_position'])
print('best_value_ever    : ',optimization.member[list(optimization.best_position_ever.keys())[0]]['best_value'])
print('best_value_obtained at iteration no: %s' % (optimization.best_value_iteration))

In [ ]:
################################################################################################
# %%
list_mean_closest  = []
list_mean_furthest = []
list_mean2target   = []
list_mean2opt      = []
for ii in range(optimization.nop):
    print('particle no: %s' % (ii))
    closest_distance   = []
    furthest_distance  = []
    distance2target    = []
    best_vals_ever = list(optimization.member[list(optimization.best_position_ever.keys())[ii]]['best_position'].values())
    for _ in range(3):
        function2opt(params=best_vals_ever)
        for key in particles.member.keys():
            closest_distance.append(np.abs(particles.member[key]['abs_distance_sorted'][1]))
            furthest_distance.append(np.abs(particles.member[key]['abs_distance_sorted'][-1]))
            distance2target.append(particles.member[key]['distance2target'])
    list_mean_closest.append(np.mean(closest_distance))
    list_mean_furthest.append(np.mean(furthest_distance))
    list_mean2target.append(np.mean(distance2target))
    list_mean2opt.append(np.mean(closest_distance) + np.mean(furthest_distance) + np.mean(distance2target))

In [ ]:
################################################################################################
# %%
for _ in range(5):
    best_vals_ever = list(optimization.member[list(optimization.best_position_ever.keys())[0]]['best_position'].values())
    function2opt(params=best_vals_ever)
    closest_distance     = np.zeros(number_of_particles)
    furthest_distance    = np.zeros(number_of_particles)
    distance2target      = np.zeros(number_of_particles)
    for ii in range(particles.nop):
        closest_distance[ii]  = np.abs(particles.member[str(ii)]['abs_distance_sorted'][1])
        furthest_distance[ii] = np.abs(particles.member[str(ii)]['abs_distance_sorted'][-1])
        distance2target[ii]   = particles.member[str(ii)]['distance2target']

In [ ]:
################################################################################################
# %%
bestfornow = list(optimization.member[list(optimization.best_position_ever.keys())[35]]['best_position'].values())
np.savetxt('best_coefficients01.txt',bestfornow)